The dataset of tabular-playground-series-feb-2021 was generated using a CTGAN based on a real dataset. (Description)

The accuracy of the machine learning model may be improved by further augmenting the provided data with CTGAN.

In [ ]:
!pip install sdv
!pip install -U pandas matplotlib seaborn

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import lightgbm as lgb
import optuna.integration.lightgbm as oplgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import category_encoders as ce
import seaborn as sns
from sdv.tabular import CTGAN

In [ ]:
df_train = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/train.csv")
df_test = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/test.csv")
df_sample = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/sample_submission.csv")

In [ ]:
df_train

In [ ]:
df_test

In [ ]:
train_id = df_train["id"]
test_id = df_test["id"]

df_train.drop("id", axis=1, inplace=True)
df_test.drop("id", axis=1, inplace=True)

# Data Augmentation

CTGAN is implemented in https://github.com/sdv-dev/SDV

In [ ]:
ctgan = CTGAN(verbose=True)

In [ ]:
sample_for_augmentation = df_train.sample(20000, random_state=2021)

In [ ]:
ctgan.fit(sample_for_augmentation)

In [ ]:
augmentation_sampled = ctgan.sample(20000)

In [ ]:
augmentation_sampled

# Comparison of distribution

In [ ]:
df_train_and_augmentation = pd.concat([sample_for_augmentation, augmentation_sampled], axis=0)

In [ ]:
is_augment = np.zeros(len(df_train_and_augmentation))
is_augment[len(sample_for_augmentation):] = 1
df_train_and_augmentation["is_augment"] = is_augment

In [ ]:
cat_features = [f"cat{i}" for i in range(9 + 1)]

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(20, 5))
axes = axes.flatten()
for cat_col, ax in zip(cat_features, axes):
    sns.countplot(data=df_train_and_augmentation, x=cat_col, hue="is_augment", ax=ax)
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(20, 5))
axes = axes.flatten()
for cat_col, ax in zip(cat_features, axes):
    sns.barplot(data=df_train_and_augmentation, x=cat_col, y="target", hue="is_augment", ax=ax)
plt.tight_layout()
plt.show()

In [ ]:
num_features = [f"cont{i}" for i in range(13 + 1)]

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=7, figsize=(20, 5))
axes = axes.flatten()
for num_col, ax in zip(num_features, axes):
    sns.violinplot(data=df_train_and_augmentation, y=num_col, x="is_augment", ax=ax)
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=7, figsize=(20, 5))
axes = axes.flatten()
for num_col, ax in zip(num_features, axes):
    sns.scatterplot(data=df_train_and_augmentation, y="target", x=num_col, hue="is_augment", alpha=0.01, ax=ax)
plt.tight_layout()
plt.show()

In [ ]:
ctgan_sampled_save = ctgan.sample(300000)

In [ ]:
ctgan_sampled_save.to_csv("ctgan_sampled_v1.csv", index=False)